In [1]:
import numpy as np
import pandas as pd
import torch
from pymatgen.io.cif import CifParser
from tqdm import tqdm
from joblib import Parallel, delayed

/mnt/c/Users/123/Desktop/2D_crystals/Band_studies/band_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_pickle('pilot/data.pickle.gz')
structures = pd.Series(data=df['initial_structure'].values, index=df['descriptor_id'].values, name='structures')

In [3]:
descriptors = pd.read_csv('descriptors.csv', index_col='_id')
descriptors = descriptors[['base', 'cell']]

In [4]:
prepared = pd.merge(structures, descriptors, left_index=True, right_index=True)
prepared

,structures,base,cell
6141cf10b842c2e72e2f2d42,[[1.27612629e-07 1.84192955e+00 3.71975100e+00...,MoS2,"[8, 8, 1]"
6141cf10cc0e69a0cf28ab33,[[1.27612629e-07 1.84192955e+00 3.71975100e+00...,MoS2,"[8, 8, 1]"
6141cf10cc0e69a0cf28ab33,[[1.27612629e-07 1.84192955e+00 3.71975100e+00...,MoS2,"[8, 8, 1]"
6141cf123ac25c70a5c6c835,[[1.27612629e-07 1.84192955e+00 3.71975100e+00...,MoS2,"[8, 8, 1]"
6141cf123ac25c70a5c6c835,[[1.27612629e-07 1.84192955e+00 3.71975100e+00...,MoS2,"[8, 8, 1]"
...,...,...,...
6141cf184e27a1844a5efff8,[[1.27612629e-07 1.84192955e+00 3.71975100e+00...,MoS2,"[8, 8, 1]"
6141cf184e27a1844a5efff8,[[1.27612629e-07 1.84192955e+00 3.71975100e+00...,MoS2,"[8, 8, 1]"
6141cf184e27a1844a5efff8,[[1.27612629e-07 1.84192955e+00 3.71975100e+00...,MoS2,"[8, 8, 1]"
6141cf184e27a1844a5efff8,[[1.27612629e-07 1.84192955e+00 3.71975100e+00...,MoS2,"[8, 8, 1]"


In [5]:
unit_cells = {
    'MoS2': CifParser("MoS2.cif").get_structures(primitive=False)[0],
    'WSe2': CifParser("WSe2.cif").get_structures(primitive=False)[0]
}

/mnt/c/Users/123/Desktop/2D_crystals/Band_studies/band_env/lib/python3.10/site-packages/pymatgen/io/cif.py:1169: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [6]:
prepared = prepared.values.tolist()
prepared = [[p[0], p[1], eval(p[2])] for p in prepared]

In [7]:
from MEGNetSparse import convert_to_sparse_representation

dataset = Parallel(n_jobs=-1)(
            delayed(convert_to_sparse_representation)(p[0], unit_cells[p[1]], p[2], True) for p in tqdm(prepared))

 16%|█████████████████████▊                                                                                                                  | 32/200 [00:37<02:34,  1.09it/s]/mnt/c/Users/123/Desktop/2D_crystals/Band_studies/band_env/lib/python3.10/site-packages/pymatgen/core/structure.py:799: UserWarning: Not all sites have property was. Missing values are set to None.
  warnings.warn(f"Not all sites have property {k}. Missing values are set to None.")
/mnt/c/Users/123/Desktop/2D_crystals/Band_studies/band_env/lib/python3.10/site-packages/pymatgen/core/structure.py:799: UserWarning: Not all sites have property was. Missing values are set to None.
  warnings.warn(f"Not all sites have property {k}. Missing values are set to None.")
 44%|███████████████████████████████████████████████████████████▊                                                                            | 88/200 [00:39<00:10, 10.31it/s]/mnt/c/Users/123/Desktop/2D_crystals/Band_studies/band_env/lib/python3.10/site-package

In [8]:
targets = pd.read_csv('pilot/targets.csv.gz')
targets = torch.tensor(targets['formation_energy_per_site'].values).float()

In [9]:
train_data, test_data = dataset[:100], dataset[100:]
train_targets, test_targets = targets[:100], targets[100:]

In [10]:
config = {
    'model': {
        'train_batch_size': 50,
        'test_batch_size': 50,
        'add_z_bond_coord': True,
        'atom_features': 'werespecies',
        'state_input_shape': 2,
        'cutoff': 10,
        'edge_embed_size': 10,
        'vertex_aggregation': 'mean',
        'global_aggregation': 'mean',
        'embedding_size': 32,
        'nblocks': 3,
    },
    'optim': {
        'factor': 0.5,
        'patience': 30,
        'threshold': 5e-2,
        'min_lr': 1e-5,
        'lr_initial': 1e-3,
        'scheduler': 'ReduceLROnPlateau',
    }
}

In [11]:
from MEGNetSparse import MEGNetTrainer

trainer = MEGNetTrainer(config, 'cpu')

In [12]:
trainer.prepare_data(train_data, train_targets, test_data, test_targets, 'formation_energy')

adding targets to data
converting data


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 930.11it/s]


In [13]:
trainer.train_one_epoch()

target: formation_energy device: cpu


(0.5600546646118164, 0.9621846)

In [14]:
trainer.evaluate_on_test()

1.0083928298950195